In [ ]:
!pip install datasets
import transformers
import datasets
from datasets import load_dataset


In [ ]:
loaded_data = load_dataset("dair-ai/emotion")
print(loaded_data)

In [ ]:
##just
print(loaded_data["train"][0])
print(loaded_data["validation"][101])
print(loaded_data["test"][0])

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenizenewdataset(loaded_data):
  return tokenizer(loaded_data["text"],padding = True,truncation = True)
newtokendata = loaded_data.map(tokenizenewdataset)
print(newtokendata)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 6)

In [ ]:
from transformers import TrainingArguments,Trainer
trainingarguments = TrainingArguments(output_dir = "test-trainer",
                                      per_device_train_batch_size = 8,
                                      per_device_eval_batch_size = 8,
                                      eval_strategy = "epoch",
                                      num_train_epochs = 3,
                                      weight_decay = 0.01,
                                      logging_dir = "./logs",
                                      learning_rate = 1e-5,
                                      load_best_model_at_end = True,
                                      save_strategy = "epoch",
                                      save_total_limit = 3,
                                      metric_for_best_model = "eval_loss",
                                      report_to = "none",
                                      )

In [ ]:
trainer = Trainer(model = model,
                  args = trainingarguments,
                  data_collator = data_collator,
                  tokenizer = tokenizer,
                  train_dataset = newtokendata["train"],
                  eval_dataset =newtokendata["validation"])
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
predictions = trainer.predict(newtokendata["test"])
print(predictions)

In [ ]:
### now converting the logits to label classes we got 6 logits per 1 row because we are classifyingto 6 classes.pick the highest score .predictions return tuples
import numpy as np
preds = np.argmax(predictions.predictions,axis = 1)

In [ ]:
label_names = loaded_data["train"].features["label"].names
print(label_names)
label_names1 = loaded_data["train"].features["label"]
print(type(label_names1))

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score,classification_report,f1_score
accuracy  = accuracy_score(preds,predictions.label_ids)
print(accuracy)
label_names = loaded_data["train"].features["label"].names
report = classification_report(preds,predictions.label_ids,target_names = label_names)
print(report)
print("F1 Score:", f1_score(predictions.label_ids, preds,average = "weighted"))

In [ ]:
##convertinglabelids to class names
# Map predicted label IDs to class names
pred_class_names = [label_names[i] for i in preds]
print(pred_class_names[:10])
